Dataset folder
https://drive.google.com/drive/folders/1o0jugzouwCNBc0ooodfDT_ViSw4QP2GR?usp=drive_link

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from datetime import timedelta
import matplotlib.pyplot as plt

london_arr = pd.read_csv('/content/london_arrivals_processed.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/content/london_arrivals_processed.csv'

In [ ]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display(london_arr)

In [ ]:
london_arr['ACTUAL ARRIVAL TIME'] = pd.to_datetime(london_arr['ACTUAL ARRIVAL TIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
london_arr = london_arr.dropna(subset=['ACTUAL ARRIVAL TIME']) # arima can't handle missing values

# there was partial data for 2023-10-01 and it affected the forecast, so I removed it
london_arr = london_arr[london_arr['ACTUAL ARRIVAL TIME'].dt.date != pd.to_datetime('2023-10-01').date()]

In [ ]:
daily_arr = london_arr.groupby(london_arr['ACTUAL ARRIVAL TIME'].dt.date).size() # how many arrivals on each date
daily_arr = pd.Series(daily_arr, index=pd.to_datetime(daily_arr.index))

print(daily_arr)

**ARIMA model**

In [ ]:
#The fitted values from ARIMA(5,0,2) closely follow the actual data, suggesting that this configuration captures short-term dynamics effectively.
plt.figure(figsize=(10,5))
plt.plot(daily_arr, label='Actual Data', color='blue')
plt.plot(model_fit.fittedvalues, label='ARIMA(5,0,2) Fitted', color='red')
plt.title('Actual vs Fitted - ARIMA(5,0,2)')
plt.ylabel('Arrivals')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
model = ARIMA(daily_arr, order=(5,0,2)) # not sure about order
model_fit = model.fit()

In [ ]:
steps = 14 # prediction for the next 14 days
forecast_dates =[]
forecast = model_fit.forecast(steps=steps)

last_date = daily_arr.index[-2] # start prediction after this date

for i in range(1, steps + 1):
    forecast_dates.append(last_date + timedelta(days=i)) # list of days we're forecasting for

forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast_Arrivals': forecast}).set_index('Date')
print(forecast_df)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(daily_arr.index, daily_arr, label='Historical Arrivals', color='blue')
plt.plot(forecast_df.index, forecast_df['Forecast_Arrivals'], label='Forecast (Next 2 Weeks)', color='red')
plt.title('Forecast of Flights to UK')
plt.ylabel('Arrivals')
plt.axvline(x=forecast_df.index[0], color='grey', linestyle='--', linewidth=1.5, label='Forecast Start')
plt.legend()
plt.show()